In [1]:
import numpy as np
import pandas as pd
import requests

In [2]:
# Base URL and headers for rated posts
url_rated_posts = "https://api.socialverseapp.com/posts/rating"
headers = {
    "Flic-Token": "flic_6e2d8d25dc29a4ddd382c2383a903cf4a688d1a117f6eb43b35a1e7fadbb84b8"
}

# Variables to handle pagination
page = 1
page_size = 1000
all_rated_posts = []  # List to store all rated posts

while True:
    # Fetch data for the current page
    params = {
        "page": page,
        "page_size": page_size,
        "resonance_algorithm": "resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if"
    }
    response = requests.get(url=url_rated_posts, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()  # Assuming the API returns JSON data
        rated_posts = data.get("posts", [])  # Adjust the key if necessary
        
        # Append rated posts to the master list
        all_rated_posts.extend(rated_posts)
        
        # Break if no more posts are returned
        if len(rated_posts) < page_size:
            break
        
        # Increment the page for the next request
        page += 1
    else:
        print(f"Error: {response.status_code}, {response.text}")
        break

# Convert to Pandas DataFrame
df_rated_posts = pd.DataFrame(all_rated_posts)


In [4]:
# Base URL and headers for posts
url_all_posts = "https://api.socialverseapp.com/posts/summary/get"
headers = {
    "Flic-Token": "flic_6e2d8d25dc29a4ddd382c2383a903cf4a688d1a117f6eb43b35a1e7fadbb84b8"
}

# Variables to handle pagination
page = 1
page_size = 1000
all_posts = []  # List to store all posts

while True:
    # Fetch data for the current page
    params = {
        "page": page,
        "page_size": page_size
    }
    response = requests.get(url=url_all_posts, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()  # Assuming the API returns JSON data
        posts = data.get("posts", [])  # Adjust the key if necessary
        
        # Append posts to the master list
        all_posts.extend(posts)
        
        # Break if no more posts are returned
        if len(posts) < page_size:
            break
        
        # Increment the page for the next request
        page += 1
    else:
        print(f"Error: {response.status_code}, {response.text}")
        break

# Convert to Pandas DataFrame
df_posts = pd.DataFrame(all_posts)

In [114]:
url_all_users = "https://api.socialverseapp.com/users/get_all"
headers = {
    "Flic-Token": "flic_6e2d8d25dc29a4ddd382c2383a903cf4a688d1a117f6eb43b35a1e7fadbb84b8"
}

# Variables to handle pagination
page = 1
page_size = 1000
all_users = []  # List to store all users

while True:
    # Fetch data for the current page
    params = {
        "page": page,
        "page_size": page_size
    }
    response = requests.get(url=url_all_users, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()  # Assuming the API returns JSON data
        users = data.get("users", [])  # Adjust the key if necessary
        
        # Append users to the master list
        all_users.extend(users)
        
        # Break if no more users are returned
        if len(users) < page_size:
            break
        
        # Increment the page for the next request
        page += 1
    else:
        print(f"Error: {response.status_code}, {response.text}")
        break

# Convert to Pandas DataFrame
df_users = pd.DataFrame(all_users)

In [115]:
df_users.rename(columns={'id':'user_id'},inplace=True)
df_posts.rename(columns={'id':'post_id'},inplace=True)

In [116]:
def convert_category_to_list(row):
    category_list = row['name']
    return category_list

In [117]:
df_posts['category']=df_posts['category'].apply(convert_category_to_list)

In [118]:
def extract_genres(data):
 if isinstance(data, dict):
        return data.get('genre', [])
 return []
df_posts['genre']=df_posts['post_summary'].apply(extract_genres)

In [119]:
def extract_emotions(data):
 if isinstance(data, dict):
        return data.get('emotions', [])
 return []

In [120]:
df_posts['mood'] = df_posts['post_summary'].apply(extract_emotions)

In [121]:
def Convert(data):
    # Check if data is a dictionary and then look for each specific key
    if isinstance(data, dict):
        # Return values based on the available key in the dictionary
        return (
            data.get('conveyed', []) or
            data.get('expressed_emotions', []) or
            data.get('emotional_content', []) or
            data.get('moods', []) or
            data.get('moods_conveyed', []) or
            data.get('conveyed_emotions', []) or
            data.get('conveyed_moods', []) or
            data.get('emotions_expressed', []) or
            data.get('described_emotions', []) or
            data.get('expressed',[]) or
            data.get('emotional_states',[]) or
            data.get('emotions_conveyed',[])
        )
    
    # If data is not a dictionary, just return it as is
    return data

In [122]:
df_posts['mood']=df_posts['mood'].apply(Convert)

In [123]:
df_posts=df_posts.dropna()

In [124]:
df_posts=df_posts[['post_id','category','title','genre','mood']]

In [125]:
df_posts

,post_id,category,title,genre,mood
0,11,Vible,Recipe for a flow state,Educational/Inspirational,"[enthusiasm, engagement, curiosity, passion]"
1,12,Vible,Why fit in..?,Animation/Emotional Narrative,"[Anger, Determination, Pain, Empowerment, Isol..."
2,13,Vible,Transcending Singularity,Philosophical Exploration,"[Wonder, Intensity, Compassion, Engagement, Co..."
3,14,Vible,Peak Performance?,Motivational/Self-help,"[excitement, passion, contemplation]"
4,15,Vible,Our Existential Situation,Philosophical Commentary,"[Concern, Passion, Reflectiveness, Urgency, Ho..."
...,...,...,...,...,...
1407,1941,Super Feed,💛🦋✨\n#universe #tamilmusic #tamilcinimas #love...,Cinematic Spiritual Experience,"[Affection, Contemplation, Reflection, Curiosi..."
1408,1942,Super Feed,Radhe Radhe ❤️🙏\n\n#gurudev #premanandjimahara...,Motivational/Inspirational,"[Determination, Passion, Encouragement, Hope]"
1409,1943,Super Feed,That feeling would be unexplainable.\n\nFollow...,Adventure/Lifestyle,"[excitement, camaraderie, enthusiasm, confidence]"
1410,1944,Super Feed,#musculação #treino #fitness #foco #evolução #...,celebratory performance,"[excitement, joy, anticipation]"


In [128]:
df_posts['genre'] = df_posts['genre'].apply(lambda x: 'Unknown' if x == [] else x)
df_posts['mood'] = df_posts['mood'].apply(lambda x: 'Unknown' if x == [] else x)

In [129]:
df_posts

,post_id,category,title,genre,mood
0,11,Vible,Recipe for a flow state,Educational/Inspirational,"[enthusiasm, engagement, curiosity, passion]"
1,12,Vible,Why fit in..?,Animation/Emotional Narrative,"[Anger, Determination, Pain, Empowerment, Isol..."
2,13,Vible,Transcending Singularity,Philosophical Exploration,"[Wonder, Intensity, Compassion, Engagement, Co..."
3,14,Vible,Peak Performance?,Motivational/Self-help,"[excitement, passion, contemplation]"
4,15,Vible,Our Existential Situation,Philosophical Commentary,"[Concern, Passion, Reflectiveness, Urgency, Ho..."
...,...,...,...,...,...
1407,1941,Super Feed,💛🦋✨\n#universe #tamilmusic #tamilcinimas #love...,Cinematic Spiritual Experience,"[Affection, Contemplation, Reflection, Curiosi..."
1408,1942,Super Feed,Radhe Radhe ❤️🙏\n\n#gurudev #premanandjimahara...,Motivational/Inspirational,"[Determination, Passion, Encouragement, Hope]"
1409,1943,Super Feed,That feeling would be unexplainable.\n\nFollow...,Adventure/Lifestyle,"[excitement, camaraderie, enthusiasm, confidence]"
1410,1944,Super Feed,#musculação #treino #fitness #foco #evolução #...,celebratory performance,"[excitement, joy, anticipation]"


In [130]:
df_posts.isnull().sum()

post_id     0
category    0
title       0
genre       0
mood        0
dtype: int64

In [131]:
df_users=df_users[['user_id','username']]

In [132]:
df_users

,user_id,username
0,1,afrobeezy
1,2,doey
2,3,doeyy
3,4,doeyyy
4,5,kinha
...,...,...
1230,1254,devrajoriya
1231,1255,naresh_09i3f7l3c
1232,1256,yuvraj_01i9l5f7c
1233,1257,master


In [133]:
rated_posts=df_rated_posts.merge(df_posts,on='post_id')

In [134]:
rated_posts

,id,post_id,user_id,rating_percent,rated_at,category,title,genre,mood
0,1,516,1,64,2024-01-11 12:45:41,Vible,,Religious / Inspirational,"[Awe, Reverence, Strength, Benevolence]"
1,2,160,1,62,2024-01-11 12:47:15,Vible,,Artistic Narrative/Short Film,"[Contemplative, Somber, Vulnerable]"
2,3,19,1,20,2024-01-11 12:47:28,Vible,WE ARE MADE OF STARS,Educational/Documentary,"[wonder, curiosity, reflection, passion]"
3,4,148,1,100,2024-01-11 12:49:35,Vible,,motivational/reflective,"[calmness, thoughtfulness, optimism, gratefuln..."
4,5,513,1,63,2024-01-11 12:50:58,Vible,,animated spiritual narrative,"[determination, strength, courage]"
...,...,...,...,...,...,...,...,...,...
2819,2980,1807,1105,27,2024-12-08 05:23:21,Vible,motivation,Educational/Instructional,"[inquisitive, analytical, neutral]"
2820,2981,546,1105,36,2024-12-08 05:23:24,Vible,,Spiritual/Devotional,"[Tranquility, Reassurance, Calmness, Strength,..."
2821,2982,1140,1105,12,2024-12-08 05:23:44,Vible,5 simple ways to make this day better. 🙏🏽#lead...,Self-help / Motivational,"[Positivity, Sincerity, Confidence, Empathy, E..."
2822,2983,692,1105,10,2024-12-08 05:23:47,Vible,Lord is near to the broken heart,Spiritual/Religious,"[longing, despair, hope, compassion, warmth, u..."


In [135]:
complete_df=rated_posts.merge(df_users,on='user_id')

In [136]:
complete_df['title']=complete_df['title'].replace('','untitled_video')

In [137]:
complete_df

,id,post_id,user_id,rating_percent,rated_at,category,title,genre,mood,username
0,1,516,1,64,2024-01-11 12:45:41,Vible,untitled_video,Religious / Inspirational,"[Awe, Reverence, Strength, Benevolence]",afrobeezy
1,2,160,1,62,2024-01-11 12:47:15,Vible,untitled_video,Artistic Narrative/Short Film,"[Contemplative, Somber, Vulnerable]",afrobeezy
2,3,19,1,20,2024-01-11 12:47:28,Vible,WE ARE MADE OF STARS,Educational/Documentary,"[wonder, curiosity, reflection, passion]",afrobeezy
3,4,148,1,100,2024-01-11 12:49:35,Vible,untitled_video,motivational/reflective,"[calmness, thoughtfulness, optimism, gratefuln...",afrobeezy
4,5,513,1,63,2024-01-11 12:50:58,Vible,untitled_video,animated spiritual narrative,"[determination, strength, courage]",afrobeezy
...,...,...,...,...,...,...,...,...,...,...
2819,2980,1807,1105,27,2024-12-08 05:23:21,Vible,motivation,Educational/Instructional,"[inquisitive, analytical, neutral]",Dylan
2820,2981,546,1105,36,2024-12-08 05:23:24,Vible,untitled_video,Spiritual/Devotional,"[Tranquility, Reassurance, Calmness, Strength,...",Dylan
2821,2982,1140,1105,12,2024-12-08 05:23:44,Vible,5 simple ways to make this day better. 🙏🏽#lead...,Self-help / Motivational,"[Positivity, Sincerity, Confidence, Empathy, E...",Dylan
2822,2983,692,1105,10,2024-12-08 05:23:47,Vible,Lord is near to the broken heart,Spiritual/Religious,"[longing, despair, hope, compassion, warmth, u...",Dylan


In [138]:
complete_df.isnull().sum()

id                0
post_id           0
user_id           0
rating_percent    0
rated_at          0
category          0
title             0
genre             0
mood              0
username          0
dtype: int64

In [139]:
complete_df.to_csv("completed_data_for_cold_start.csv")

In [145]:
def Recommend(moods):
    """
    Recommend posts based on one or more moods. The moods column contains lists of moods.

    :param moods: A list of moods (or a single mood as a string) to filter recommendations.
    :return: A DataFrame of recommended posts.
    """
    # Ensure moods is a list
    if isinstance(moods, str):
        moods = [moods]

    # Filter the DataFrame for rows where any mood in the 'moods' column matches one of the provided moods
    filtered_df = complete_df[complete_df['mood'].apply(lambda x: any(mood in x for mood in moods))]

    # Optionally, sort by 'rating_percent' if available
    recommendations = filtered_df.sort_values(ascending=False, by='rating_percent')

    # Remove duplicates based on 'title' column to avoid recommending the same post multiple times
    recommendations = recommendations.drop_duplicates(subset='title')

    return recommendations

In [146]:
Recommend(['Awe', 'Reverence', 'Strength', 'Benevolence'])

,id,post_id,user_id,rating_percent,rated_at,category,title,genre,mood,username
1690,1782,165,908,100,2024-08-16 20:51:28,Vible,untitled_video,Artistic narrative exploration,"[Seriousness, Contemplation, Urgency, Reverence]",daksh_bhardwaj
658,692,775,187,99,2024-06-03 11:37:07,Vible,part 2,Religious Educational,"[Reverence, Intrigue, Anticipation, Fear, Brav...",mridul
2732,2869,1387,16,61,2024-12-05 15:35:57,Bloom Scroll,"Our solar system is on an incredible journey, ...",Educational / Science,"[Wonder, Incredulity, Awe]",michaeldadziie
1390,1478,820,1,57,2024-07-23 12:13:50,Vible,"As I live, says the Lord, every knee shall bow...",Religious/Inspirational,"[Anticipation, Sorrow, Hope, Resilience, Rever...",afrobeezy
2347,2471,1331,1076,30,2024-10-31 16:49:20,Flic,The perfect conditions in Norway .Would you ta...,Nature Documentary,"[Tranquility, Gratitude, Awe]",adarsh
1634,1726,1106,1,28,2024-08-13 16:47:42,Vible,Understanding what it is can make it change dr...,Educational/Inspirational,"[Empathy, Concern, Reassurance, Vulnerability,...",afrobeezy
